# Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import os
import cv2 as cv

In [2]:
# params 

original_image_size = (720,1280,3)

# Define the model

In [3]:
# size of the input images
input_image_size = (416,416,3)

# bounding boxes per cell
BB_per_cell = 3 

# number of classes in the dataset
Classes = 5

# Batch size
batch_ = 16

In [4]:
# Weight matrix initialization whenever using relu
initializer_ = tf.keras.initializers.he_normal()

In [5]:
def Build_model(initializer_, input_image_size, BB_per_cell, Classes, batch_, model_name):
    
    # input lauer
    input_layer = tf.keras.layers.Input(shape=input_image_size, batch_size=batch_, name="input")

    # down_sampling
    batch_norm_1 = tf.keras.layers.BatchNormalization()(input_layer)
    
    conv_1 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1),
                 activation='relu', kernel_initializer=initializer_, padding="same", name="conv_1")(batch_norm_1)

    pooling_1 = tf.keras.layers.MaxPool2D(pool_size=(2,2), name="pool_1")(conv_1)
    
    batch_norm_1 = tf.keras.layers.BatchNormalization()(pooling_1)

    conv_2 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides=(1,1),
                 activation='relu', kernel_initializer=initializer_,padding="same", name="conv_2")(batch_norm_1)
    
    pooling_2 = tf.keras.layers.MaxPool2D(pool_size=(2,2), name="pool_2")(conv_2)
    
    batch_norm_2 = tf.keras.layers.BatchNormalization()(pooling_2)

    conv_3 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1),
                 activation='relu', kernel_initializer=initializer_,padding="same", name="conv_3")(batch_norm_2)
    
    pooling_3 = tf.keras.layers.MaxPool2D(pool_size=(2,2), name="pool_3")(conv_3)
    
    batch_norm_3 = tf.keras.layers.BatchNormalization()(pooling_3)

    conv_4 = tf.keras.layers.Conv2D(filters=BB_per_cell*(5+Classes), kernel_size=(3,3), strides=(1,1),
                 activation=None, padding="same", name="detection_conv_4")(batch_norm_3)
    
    return tf.keras.Model(inputs=input_layer, outputs=[conv_4], name=model_name)

In [6]:
def mini_conv_block(input_, filters, kernel_size, strides, activation, ker_initializer, padd_):
    batch_norm = tf.keras.layers.BatchNormalization()(input_)
    conv = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides,
                 activation=activation, kernel_initializer=ker_initializer, padding=padd_)(batch_norm)
    pooling = tf.keras.layers.MaxPool2D(pool_size=(2,2))(conv)
    return pooling 

In [7]:
def Build_model_(initializer_, input_image_size, BB_per_cell, Classes, batch_, model_name):
    
    # input lauer
    input_layer = tf.keras.layers.Input(shape=input_image_size, batch_size=batch_, name="input")

    first_mini_block = mini_conv_block(input_layer, filters=512, kernel_size=(3,3), 
                       strides=(1,1), activation="relu", ker_initializer=initializer_, padd_="same")

    second_mini_block = mini_conv_block(first_mini_block, filters=256, kernel_size=(3,3), 
                       strides=(1,1), activation="relu", ker_initializer=initializer_, padd_="same")
    
    third_mini_block = mini_conv_block(second_mini_block, filters=128, kernel_size=(3,3), 
                       strides=(1,1), activation="relu", ker_initializer=initializer_, padd_="same")
    
    fourth_mini_block = mini_conv_block(third_mini_block, filters=64, kernel_size=(3,3), 
                       strides=(1,1), activation="relu", ker_initializer=initializer_, padd_="same")
    
    fifth_mini_block = mini_conv_block(fourth_mini_block, filters=64, kernel_size=(3,3), 
                       strides=(1,1), activation="relu", ker_initializer=initializer_, padd_="same")
    
    batch_norm_3 = tf.keras.layers.BatchNormalization()(fifth_mini_block)

    conv_4 = tf.keras.layers.Conv2D(filters=BB_per_cell*(5+Classes), kernel_size=(3,3), strides=(1,1),
                 activation=None,padding="same")(batch_norm_3)
    
    return tf.keras.Model(inputs=input_layer, outputs=[conv_4], name=model_name)

In [8]:
model_ = Build_model_(initializer_, input_image_size, BB_per_cell, Classes, batch_, "My_SSD_Detector_")

In [9]:
model = Build_model(initializer_, input_image_size, BB_per_cell, Classes, batch_, "My_SSD_Detector")

In [10]:
model.summary()

Model: "My_SSD_Detector"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(16, 416, 416, 3)]       0         
_________________________________________________________________
batch_normalization_6 (Batch (16, 416, 416, 3)         12        
_________________________________________________________________
conv_1 (Conv2D)              (16, 416, 416, 256)       7168      
_________________________________________________________________
pool_1 (MaxPooling2D)        (16, 208, 208, 256)       0         
_________________________________________________________________
batch_normalization_7 (Batch (16, 208, 208, 256)       1024      
_________________________________________________________________
conv_2 (Conv2D)              (16, 208, 208, 128)       295040    
_________________________________________________________________
pool_2 (MaxPooling2D)        (16, 104, 104, 128)   

In [11]:
model_.summary()

Model: "My_SSD_Detector_"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(16, 416, 416, 3)]       0         
_________________________________________________________________
batch_normalization (BatchNo (16, 416, 416, 3)         12        
_________________________________________________________________
conv2d (Conv2D)              (16, 416, 416, 512)       14336     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (16, 208, 208, 512)       0         
_________________________________________________________________
batch_normalization_1 (Batch (16, 208, 208, 512)       2048      
_________________________________________________________________
conv2d_1 (Conv2D)            (16, 208, 208, 256)       1179904   
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (16, 104, 104, 256)  